In [1]:
import pickle
from snlg_tools import group_by_event,merge_pkl_list
import glob

file_list=glob.glob("./Inputs/snlg_list_*.pkl")
print(file_list)

snlg_list = merge_pkl_list(file_list, output_path=None,drop_waveforms=True)

snlg_byevent = group_by_event(snlg_list)

with open("./Inputs/byevent_snlg_iris.pkl", "wb") as f:
    pickle.dump(snlg_byevent,f)

['./Inputs/snlg_list_iris_0.pkl']
./Inputs/snlg_list_iris_0.pkl done.


# For parallel code output

In [2]:
# import pickle
# from snlg_tools import group_by_event,merge_pkl_list
# import glob

# file_list_iris=glob.glob("./Indian_Parallel/worker_*/snlg_list_iris*.pkl")
# file_list = file_list_iris

# print(len(file_list))

# snlg_list = merge_pkl_list(file_list, output_path=None,drop_waveforms=True)

# with open("./Outputs/raw_snlg_esdc_iris.pkl", "wb") as f:
#     pickle.dump(snlg_list,f)

# snlg_byevent = group_by_event(snlg_list)

# with open("./Outputs/byevent_snlg_esdc_iris.pkl", "wb") as f:
#     pickle.dump(snlg_byevent,f)

# Plot

In [3]:
# Plot by event
import matplotlib.pyplot as plt
import numpy as np
import os 
from obspy import UTCDateTime
import pandas as pd

fig_dir = './Outputs'
os.makedirs(f"./{fig_dir}",exist_ok=True)

time_before_p = 60
snlg_records = []

for event_time, pairs in snlg_byevent.items():
    # filter out any None values
    event_snlg_list = [p for p in pairs if p.SnRLg_raw is not None and not np.isnan(p.SnRLg_raw)]
    if not event_snlg_list:
        continue

    epi_dis = [one_snlg.dist for one_snlg in event_snlg_list]

    plt.figure(figsize=[10, 12])
    plt.xlabel("Time (s)",fontsize=14)
    plt.ylabel("Epicentral Distance (km)",fontsize=14)
    plt.xlim([20, 550])
    dis_min = min(epi_dis)
    dis_max = max(epi_dis)
    padding = 100
    bin_width = 20
    plt.ylim([dis_min-padding, dis_max+padding])
    step = ((dis_max - dis_min) + 2 * padding) / bin_width

    for j in np.argsort(epi_dis):
        one_snlg_now = event_snlg_list[j]
        if one_snlg_now.SnRLg_raw:
            ####################################################################
            #  origin_time       trace_start_time   p_arrival  Sn_window_start #
            #       |-------------------|--------------|-------------|-----    #
            #       ^****************** Sn_start ********************^         #
            #       ^****** shift ******^                                      #
            #                           ^**** for Sn_start index ****^         #
            ####################################################################
            stnm = f"{one_snlg_now.knetwk}.{one_snlg_now.kstnm}"
            dist = one_snlg_now.dist
            tr = one_snlg_now.t_comp_waveform_for_sn # Can be modified
            tr_lg = one_snlg_now.t_comp_waveform_for_lg # Can be modified
            dt = tr.meta.delta

            starttime = UTCDateTime(tr.stats.starttime)
            shift = starttime - one_snlg_now.origin_time
        
            Sn_start, Sn_end = one_snlg_now.sn_window
            Lg_start, Lg_end = one_snlg_now.lg_window

            Lg = tr_lg.data[round((Lg_start-shift)/dt):round((Lg_end-shift)/dt)]
            Sn = tr.data[round((Sn_start-shift)/dt):round((Sn_end-shift)/dt)]
            if one_snlg_now.SnRLg_err > 1.5 or one_snlg_now.SNR<2 or abs(one_snlg_now.SnRLg_raw) > 3:
                color = '#999999'
            else:
                color = 'k'

            plt.plot(np.arange(len(tr.data))*dt + shift, tr.data/np.abs(np.max(tr.data)) * bin_width + dist, color=color, linewidth=1)
            # Test Normalization
            plt.plot(Lg_start + np.arange(len(Lg))*dt, Lg/np.abs(np.max(tr.data)) * bin_width + dist, color='c', linewidth=1)
            plt.plot(Sn_start + np.arange(len(Sn))*dt, Sn/np.abs(np.max(tr.data)) * bin_width + dist, color='r', linewidth=1)

            p_time_plot = one_snlg_now.p_arrival - one_snlg_now.origin_time
            plt.plot([p_time_plot, p_time_plot], [dist - step / 2, dist + step / 2], 'b')
            plt.plot([one_snlg_now.a_sn, one_snlg_now.a_sn], [dist - step / 2, dist + step / 2], 'b')
            plt.plot([one_snlg_now.a_lg, one_snlg_now.a_lg], [dist - step / 2, dist + step / 2], 'b')
            
            plt.text(550, dist, f"{stnm} {one_snlg_now.SnRLg_raw:.2f}±{one_snlg_now.SnRLg_err:.2f} SNR{one_snlg_now.SNR:.1f}", color='orange', fontsize=14, fontweight='bold')
            # plt.text(40 , dist, f"{one_snlg_now.SnRLg_raw:.2f}±{one_snlg_now.SnRLg_err:.2f}", color='orange', fontsize=14, fontweight='bold')

            # Find moho depth
            if one_snlg_now.moho:
                moho_depth = f"{one_snlg_now.moho[0]:.1f}"
            else:
                moho_depth = "N/A"
            
            # Prepare a spreadsheet
            if not(one_snlg_now.SnRLg_err > 1.5 or one_snlg_now.SNR<2 or abs(one_snlg_now.SnRLg_raw) > 3):
                snlg_records.append({
                "event_time": UTCDateTime(event_time),
                "event_mag": one_snlg_now.mag,
                "event_dep": one_snlg_now.evdp,
                "moho_dep": moho_depth,
                "station": f"{one_snlg_now.knetwk}.{one_snlg_now.kstnm}",
                "distance": one_snlg_now.dist,
                "SnRLg_ratio": one_snlg_now.SnRLg_raw,
                "SnRLg_error": one_snlg_now.SnRLg_err
                })

    # plt.title(str(event) + ' dep: '+str(one_snlg_now.evdp)+' km')
    
    # Find moho depth again
    if one_snlg_now.moho:
        moho_depth = f"{one_snlg_now.moho[0]:.1f}"
    else:
        moho_depth = "N/A"
    
    plt.title(f"{event_time} | Mag: {one_snlg_now.mag:.1f} | Catalog Depth: {one_snlg_now.evdp:.1f} | Moho Depth: {moho_depth}",fontsize=14,fontweight='bold')
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{fig_dir}/'+str(event_time)+'_traces.png')
    plt.close()
    
    # Output spreadsheet
    df = pd.DataFrame(snlg_records)
    df.to_csv("Outputs/Ratios.csv", index=False)


In [4]:
import os
import numpy as np
import pygmt

# ——————————————————————————————————————————————————————————————
#  1) Setup output directory
# ——————————————————————————————————————————————————————————————
fig_dir = "./Outputs"
os.makedirs(fig_dir, exist_ok=True)

# ——————————————————————————————————————————————————————————————
#  2) Loop over events
# ——————————————————————————————————————————————————————————————
for event_time, pairs in snlg_byevent.items():
    # filter out any None values
    event_snlg = [p for p in pairs if p.SnRLg_raw is not None and p.SnRLg_err <= 1.5 and p.SNR>=2 and abs(p.SnRLg_raw) <= 3]
    if not event_snlg:
        continue

    # extract event & station coords + SnRLg
    evlo, evla = event_snlg[0].evlo, event_snlg[0].evla
    stlos = [p.stlo for p in event_snlg]
    stlas = [p.stla for p in event_snlg]
    snlgs = [p.SnRLg_raw for p in event_snlg]
    stnms = [f"{p.knetwk}.{p.kstnm}" for p in event_snlg]

    # define map region with a bit of padding
    padding = 1.5
    lon_min, lon_max = min(evlo, *stlos), max(evlo, *stlos)
    lat_min, lat_max = min(evla, *stlas), max(evla, *stlas)
    region = [lon_min - padding, lon_max + padding,
              lat_min - padding, lat_max + padding]

    # ——————————————————————————————————————————————————————————————
    #  3) Load topography & make CPT
    # ——————————————————————————————————————————————————————————————
    grid = pygmt.datasets.load_earth_relief(resolution="03m", region=region)

    # ——————————————————————————————————————————————————————————————
    #  4) Start figure
    # ——————————————————————————————————————————————————————————————
    fig = pygmt.Figure()
    pygmt.config(MAP_TICK_LENGTH_PRIMARY="5p")

    # push the map down/right so colorbar inset doesn’t overlap
    fig.shift_origin(xshift="2c", yshift="4c")
    with pygmt.config(MAP_FRAME_TYPE="fancy"):
        fig.basemap(region=region, projection="M15c",
                    frame=["xa1f0.5", "ya1f0.5", f"+t{event_snlg[0].origin_time}"])

    # draw shaded topography
    fig.grdimage(grid=grid, cmap="terra")
    
    # build your continuous diverging CPT from -2 to 2
    pygmt.makecpt(cmap="vik", series=[-2, 2, 0.01], continuous=True)

    # ——————————————————————————————————————————————————————————————
    #  5) Build & draw colored line‐segments
    #     Format = [lon, lat, z] with NaN row between segments
    # ——————————————————————————————————————————————————————————————
    lines = []
    for stlo, stla, z in zip(stlos, stlas, snlgs):
        lines.append([evlo, evla, z])
        lines.append([stlo, stla, z])
        lines.append([np.nan, np.nan, np.nan])  # segment break
    line_array = np.array(lines)

    # # === Plot the colored lines using the 3rd column as color ===
    fig.plot(data=line_array, pen="0.1p", cmap="vik", fill="+z")

    for stlo, stla, z in zip(stlos, stlas, snlgs):
        fig.plot(x=[evlo,stlo],y=[evla,stla],pen="4p,white")
        fig.plot(x=[evlo,stlo],y=[evla,stla],cmap=True,zvalue=z,pen="3p,+z")

    # === Add colorbar based on new CPT ===
    fig.colorbar(
        cmap="vik",
        position="JMR+o1.2c/0c+w10c/0.5c+e",
        frame=["a", "x+l ln(Sn/Lg)"],
        transparency=20,
    )

    # ——————————————————————————————————————————————————————————————
    #  6) Plot event & stations + labels
    # ——————————————————————————————————————————————————————————————
    fig.plot(x=[evlo], y=[evla],
             style="c0.5c", fill="white",
             pen="1.5p,black")
    fig.plot(x=stlos, y=stlas,
             style="t0.5c", fill="white",
             pen="1.5p,black")
    fig.text(x=stlos, y=stlas, text=stnms,
             font="10p,Helvetica-Bold,red",
             justify="CM")

    # ——————————————————————————————————————————————————————————————
    #  7) Save
    # ——————————————————————————————————————————————————————————————
    outfile = os.path.join(fig_dir, f"{event_time}_map.png")
    fig.savefig(outfile)
    del fig
